## Wunderground Geolookup

This notebook queries the wunderground api for station ids within a 40 km range of Florence, Stevensville Montana.  This information is then used to query weather data for each station.

#### api reference page :
https://www.wunderground.com/weather/api/d/docs?d=resources/code-samples

In [4]:
from urllib.request import urlopen
import json
import pandas as pd

In [5]:
private_key = 'x'

The following api calls return all weather stations within 40km of 'MT/town.json'

In [6]:
main_call = 'http://api.wunderground.com/api/' + private_key + '/geolookup/conditions/q/'
hamilton_query = 'MT/Hamilton.json'
florence_query = 'MT/Florence.json'
stevi_query = 'MT/Stevensville.json'

In [7]:
def api_call(url):
    f = urlopen(url)
    json_string = f.read() 
    parsed_json = json.loads(json_string)
    return parsed_json

In [8]:
florence_json = api_call(main_call + florence_query)
hamilton_json = api_call(main_call + hamilton_query)
stevi_json = api_call(main_call + stevi_query)

In [9]:
nearby_florence = florence_json['location']['nearby_weather_stations']
nearby_hamilton = hamilton_json['location']['nearby_weather_stations']
nearby_stevi = stevi_json['location']['nearby_weather_stations']

In [10]:
len(nearby_stevi['pws']['station'])

26

In [11]:
airports = []

In [12]:
airports.append(nearby_hamilton['airport']['station'][1])
airports.append(nearby_hamilton['airport']['station'][3])

In [13]:
airports

[{'city': 'Missoula',
  'country': 'US',
  'icao': 'KMSO',
  'lat': '46.91999817',
  'lon': '-114.08999634',
  'state': 'MT'},
 {'city': 'Lowell',
  'country': 'US',
  'icao': 'KP69',
  'lat': '46.13999939',
  'lon': '-115.59999847',
  'state': 'ID'}]

In [14]:
florence_pws = nearby_florence['pws']['station']
stevi_pws = nearby_stevi['pws']['station']
hamilton_pws = nearby_hamilton['pws']['station']

In [15]:
broot_pws = florence_pws + hamilton_pws + stevi_pws

In [16]:
broot_pws[0]

{'city': 'Florence',
 'country': 'US',
 'distance_km': 1,
 'distance_mi': 0,
 'id': 'KMTFLORE12',
 'lat': 46.642807,
 'lon': -114.084084,
 'neighborhood': 'Florence west side',
 'state': 'MT'}

In [17]:
# put list of dictionaries into a dataframe
df = pd.DataFrame(broot_pws)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 9 columns):
city            68 non-null object
country         68 non-null object
distance_km     68 non-null int64
distance_mi     68 non-null int64
id              68 non-null object
lat             68 non-null float64
lon             68 non-null float64
neighborhood    68 non-null object
state           68 non-null object
dtypes: float64(2), int64(2), object(5)
memory usage: 4.9+ KB


In [19]:
df = df.drop_duplicates(subset='id', keep='first')

In [20]:
df = df.sort_values(by=['city']).reset_index(drop=True)

In [22]:
df.head()

,city,country,distance_km,distance_mi,id,lat,lon,neighborhood,state
0,Corvallis,US,9,5,KMTCORVA9,46.277557,-114.045059,Hamilton Heights,MT
1,Corvallis,US,11,6,KMTCORVA12,46.323433,-114.058067,Soft Rock,MT
2,Corvallis,US,9,5,KMTCORVA21,46.301487,-114.061974,Willow Creek Rd,MT
3,Corvallis,US,7,4,KMTCORVA15,46.314114,-114.119720,Corvallis - West Edge of Town,MT
4,Corvallis,US,5,3,KMTCORVA10,46.284927,-114.106499,All View Lane,MT


In [23]:
# stations within 40km
import folium

station_map = folium.Map(
    width=900, height=900,
    location=[46.574031, -114.006500],
    zoom_start=9,
    tiles='Stamen Terrain'
)

for index, row in df.iterrows():
    popup_text = row['neighborhood'] + '</br>' + row['id']
    marker = folium.Marker(location=[row['lat'], row['lon']], popup=popup_text)
    marker.add_to(station_map)

station_map

In [123]:
# output complete lookup csv
dirname = '../output/'
filename = 'geolookup-mso_stev_ham.csv'
outputfile = dirname + filename

# df.to_csv(outputfile, index=False)

In [24]:
ids = df['id']
ids

0      KMTCORVA9
1     KMTCORVA12
2     KMTCORVA21
3     KMTCORVA15
4     KMTCORVA10
5     KMTCORVA14
6      KMTFLORE7
7      KMTFLORE4
8     KMTFLORE12
9      KMTFLORE5
10    KMTFLORE11
11    KMTFLORE17
12     KMTFLORE2
13     KMTFLORE3
14    KMTHAMIL22
15    KMTHAMIL12
16    KMTHAMIL21
17    KMTHAMIL16
18    KMTHAMIL13
19      KMTLOLO6
20      KMTLOLO7
21      KMTLOLO8
22    KMTMISSO62
23    KMTMISSO55
24    KMTMISSO44
25    KMTMISSO50
26    KMTMISSO41
27    KMTMISSO15
28    KMTMISSO87
29     KMTSTEVE8
30    KMTSTEVE13
31    KMTSTEVE17
32    KMTSTEVE19
33    KMTSTEVE12
34    KMTVICTO12
35    KMTVICTO11
Name: id, dtype: object

In [133]:
# output station IDs
dirname = '../output/'
filename = 'station_id-list.csv'

outputfile = dirname + filename
ids.to_csv(outputfile, index=False)